# Train The Cumbersome Model

In [1]:
import numpy as np

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import Adam
from keras.datasets import mnist

Using TensorFlow backend.


In [2]:
batch_size = 128
num_classes = 10
epochs = 30

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

60000 train samples
10000 test samples


In [ ]:
model = Sequential()
model.add(Dense(1200, activation='relu', input_shape=(784,)))
model.add(Dropout(0.5))
model.add(Dense(1200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

In [ ]:
error = 0
y_pred = model.predict(x_test, verbose=0)

for i in range(len(y_pred)):
    if not y_pred[i].argmax() == y_test[i].argmax():
        error += 1
print('Number of error: ', error, '/', len(y_pred))

In [ ]:
model.save_weights('cumbersome_weights.h5')
model.save('cumbersome_model.h5')

# Make Soft Target

In [3]:
from keras.models import load_model
model = load_model('cumbersome_model.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 1200)              942000    
_________________________________________________________________
dropout_5 (Dropout)          (None, 1200)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 1200)              1441200   
_________________________________________________________________
dropout_6 (Dropout)          (None, 1200)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                12010     
_________________________________________________________________
activation_2 (Activation)    (None, 10)                0         
Total params: 2,395,210
Trainable params: 2,395,210
Non-trainable params: 0
_________________________________________________________________


In [4]:
error = 0
y_pred = model.predict(x_test, verbose=0)

for i in range(len(y_pred)):
    if not y_pred[i].argmax() == y_test[i].argmax():
        error += 1
print('Number of error: ', error, '/', len(y_pred))

Number of error:  148 / 10000


In [5]:
model.pop()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 1200)              942000    
_________________________________________________________________
dropout_5 (Dropout)          (None, 1200)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 1200)              1441200   
_________________________________________________________________
dropout_6 (Dropout)          (None, 1200)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                12010     
Total params: 2,395,210
Trainable params: 2,395,210
Non-trainable params: 0
_________________________________________________________________


In [6]:
from keras.layers import Lambda
temperature = 5
model.add(Lambda(lambda x: x/temperature))
model.add(Activation('softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 1200)              942000    
_________________________________________________________________
dropout_5 (Dropout)          (None, 1200)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 1200)              1441200   
_________________________________________________________________
dropout_6 (Dropout)          (None, 1200)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                12010     
_________________________________________________________________
lambda_1 (Lambda)            (None, 10)                0         
_________________________________________________________________
activation_1 (Activation)    (None, 10)                0         
Total para

In [ ]:
soft_target = model.predict_on_batch(x_train)
np.save('soft_target.npy', soft_target)